In [ ]:
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras import regularizers
import tensorflow_addons as tfa
import numpy as np
import time
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

In [ ]:
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
np.random.seed(seed)

import logging
import gc
import random
random.seed(seed)

# Data Loading

In [ ]:
telescope = 18
batch_size = 128
epochs = 200
window = 200
stride = 50

In [ ]:
data = np.load("../input/time-series-anndl/training_data.npy")
categories = np.load("../input/time-series-anndl/categories.npy")
valid_periods = np.load("../input/time-series-anndl/valid_periods.npy")

In [ ]:
def build_sequences(target_data, valid_periods, window=200, stride=50, telescope=18):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    for i, signal in enumerate(target_data):
        #remove all initial zeros
        temp_sig = signal[valid_periods[i][0]:]
        padding_check = (len(temp_sig)-telescope)%window
        if(padding_check != 0):
            # Compute padding length
            padding_len = window - len(temp_sig)%window
            padding = np.zeros((padding_len), dtype='float32')
            temp_sig = np.concatenate((padding,temp_sig))
            assert len(temp_sig) % window == 0

        for j in np.arange(0,len(temp_sig)-window-telescope,stride):
            dataset.append(temp_sig[j:j+window])
            labels.append(temp_sig[j+window:j+window+telescope])
    return np.expand_dims(np.array(dataset),axis=-1), np.expand_dims(np.array(labels),axis=-1)

In [ ]:
X, y = build_sequences(data,valid_periods,window=window,stride=stride,telescope=telescope)
del data
X.shape, y.shape

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
del X,y
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]

# 1D ResNet + Attention

In [ ]:
def build_resnet(input_shape, output_shape):
    input_layer = tfk.Input(input_shape)
    
    # Reshape to add the feature/channel dimension
    #input_layer = tfk.layers.Reshape((200, 1))(input_layer)
        
    n_feature_maps = 64

    #input_layer = keras.layers.Input(input_shape)

    # BLOCK 1

    conv_x = tfkl.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
    conv_x = tfkl.BatchNormalization()(conv_x)
    conv_x = tfkl.Activation('relu')(conv_x)

    conv_y = tfkl.Conv1D(filters=n_feature_maps, kernel_size=5, padding='same')(conv_x)
    conv_y = tfkl.BatchNormalization()(conv_y)
    conv_y = tfkl.Activation('relu')(conv_y)

    conv_z = tfkl.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
    conv_z = tfkl.BatchNormalization()(conv_z)

    # expand channels for the sum
    shortcut_y = tfkl.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
    shortcut_y = tfkl.BatchNormalization()(shortcut_y)

    output_block_1 = tfkl.add([shortcut_y, conv_z])
    output_block_1 = tfkl.Activation('relu')(output_block_1)

    # BLOCK 2

    conv_x = tfkl.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_1)
    conv_x = tfkl.BatchNormalization()(conv_x)
    conv_x = tfkl.Activation('relu')(conv_x)

    conv_y = tfkl.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
    conv_y = tfkl.BatchNormalization()(conv_y)
    conv_y = tfkl.Activation('relu')(conv_y)

    conv_z = tfkl.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
    conv_z = tfkl.BatchNormalization()(conv_z)

    # expand channels for the sum
    shortcut_y = tfkl.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
    shortcut_y = tfkl.BatchNormalization()(shortcut_y)

    output_block_2 = tfkl.add([shortcut_y, conv_z])
    output_block_2 = tfkl.Activation('relu')(output_block_2)

    attention_probs = tfk.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same', activation='softmax')(output_block_2)
    attention_mul = tfk.layers.Multiply()([output_block_2, attention_probs])
    
    gap_layer = tfk.layers.GlobalAveragePooling1D()(attention_mul)
    output_layer = tfk.layers.Dense(output_shape[0], activation='gelu')(gap_layer)

    model = tfk.models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.AdamW())
    return model

# Training

In [ ]:
def make_dataset(X, y, batch_size=128, prefetch_amt=tf.data.experimental.AUTOTUNE):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    # reshape x from (200) to (200,1) and y from (9) to (9,1)
    dataset = dataset.map(lambda x, y: (tf.reshape(x, (200,1)), tf.reshape(y, (18,1))),num_parallel_calls=tf.data.AUTOTUNE)
    #dataset = dataset.shuffle(buffer_size=1024).batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size, drop_remainder=False)
    dataset = dataset.cache()
    dataset = dataset.prefetch(prefetch_amt)
    return dataset

In [ ]:
from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
tf.function(jit_compile=True) 
tf.keras.backend.floatx()

In [ ]:
model = build_resnet(input_shape, output_shape)
model.summary()
tfk.utils.plot_model(model, expand_nested=True, show_shapes=True)

In [ ]:
history = model.fit(
    make_dataset(X_train, y_train, batch_size=batch_size),
    validation_data=make_dataset(X_val, y_val, batch_size=batch_size),
    batch_size = batch_size,
    epochs = epochs,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', min_delta=5e-6, patience=7, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.1)
    ]
).history

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(history['val_loss'], label='Custom ResNet', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('MSE')
plt.grid(alpha=.3)

plt.show()

In [ ]:
model.save("/kaggle/working/ResNet_Custom")

In [ ]:
!zip -r Model.zip /kaggle/working/ResNet_Custom

In [ ]:
from IPython.display import FileLink
FileLink(r'Model.zip')